# 3.根據顧客的狀態(目標欄位)建立其相關的決策樹及規則，並分析及評估決策樹的效能等相關指標(例如:正確率等)。
客戶狀態

Stayed 已留下

Churned 已流失

Joined 已加入

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

from sklearn.tree import DecisionTreeClassifier ,DecisionTreeRegressor    # 決策樹
from sklearn.model_selection import train_test_split ,StratifiedKFold ,cross_val_score ,GridSearchCV ,RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder ,MinMaxScaler ,OneHotEncoder ,StandardScaler
from sklearn.metrics import classification_report ,confusion_matrix ,accuracy_score ,precision_score, recall_score, f1_score # 模型評估
from mlxtend.frequent_patterns import apriori ,association_rules
from xgboost import XGBClassifier ,XGBRegressor

sns.set(style="whitegrid")
pd.set_option('display.max_columns', None)
pd.set_option('display.unicode.ambiguous_as_wide', True)
pd.set_option('display.unicode.east_asian_width', True)
plt.rcParams['axes.unicode_minus'] = False # 正常顯示負號

In [ ]:
# Colab 進行matplotlib繪圖時顯示繁體中文
# 下載台北思源黑體並命名taipei_sans_tc_beta.ttf，移至指定路徑
!wget -O TaipeiSansTCBeta-Regular.ttf https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.font_manager import fontManager

# 改style要在改font之前
# plt.style.use('seaborn')

fontManager.addfont('TaipeiSansTCBeta-Regular.ttf')
mpl.rc('font', family='Taipei Sans TC Beta')

# EDA

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/大三/上學期/大數據決策/期末報告/customer_data_handled.csv')

df

# Processing Data

In [ ]:
df.drop(['客戶編號' ,'城市' ,'郵遞區號' ,'緯度' ,'經度'], axis=1, inplace=True)

df

讀取處理過的檔案，然後刪除用不到欄位，像是客戶編號或是城市
因為客戶編號的unique 顯示 6923，這代表對於每個樣本，客戶編號都是唯一的，這種情況下，這個特徵對於機器學習模型而言可能沒有區分度
再來是城市的unique 顯示 1106，進行One-Hot Encoding時會大幅度增加我們Feature的數量，這些城市會產生大量的0，可能會導致時間上的浪費和效能問題的產生

# 特徵工程(Feature Engineering)

In [ ]:
# 標籤編碼(LabelEncoder)
label_encoder = LabelEncoder()
labels = ['性別' ,'婚姻' ,'電話服務' ,'多線路服務' ,'網路服務' ,'線上安全服務' ,'線上備份服務' ,'設備保護計劃' ,'技術支援計劃' ,'電視節目' ,'電影節目' ,'音樂節目' ,'無限資料下載' ,'無紙化計費']
for label in labels:
    label_encoder = LabelEncoder()
    label_encoder.fit(df[label])
    df[label] = label_encoder.transform(df[label])

In [ ]:
cols = ['優惠方式' ,'網路連線類型','合約類型' ,'支付帳單方式' ,'客戶狀態','客戶流失類別' ,'客戶離開原因']
# df = pd.get_dummies(df, columns=cols)
# One-Hot Encoding 會大幅度增加我們Feature的數量、data裡面出現非常大量的0，這樣的情況在計算會時很耗時，這裡就不做了，改作LabelEncoder

for label in cols:
    label_encoder = LabelEncoder()
    label_encoder.fit(df[label])
    df[label] = label_encoder.transform(df[label])

「優惠方式」、「合約類型」這類多值的特徵使用One-Hot Encoding 會增加特徵的數量，在後續的熱力圖中也不好觀察相關性，在這種情況下，可以考慮使用其他編碼方法，而經過討論過後我們決定同樣使用LabelEncoder來處理

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/大三/上學期/大數據決策/期末報告/customer_data_handled.csv')
df.drop(['客戶編號' ,'城市' ,'郵遞區號' ,'緯度' ,'經度'], axis=1, inplace=True)

use_columns = ['性別' ,'婚姻' ,'電話服務' ,'多線路服務' ,'網路服務' ,'線上安全服務' ,'線上備份服務' ,'設備保護計劃' ,'技術支援計劃' ,'電視節目' ,'電影節目' ,'音樂節目' ,'無限資料下載' ,'無紙化計費' ,'優惠方式' ,'網路連線類型','合約類型' ,'支付帳單方式' ,'客戶狀態','客戶流失類別' ,'客戶離開原因']
df_copy = df.copy()

label_encoder = LabelEncoder()

for col in use_columns:
    df_copy[col + '_encoded'] = label_encoder.fit_transform(df[col])

    original_values = df_copy[col].unique()
    encoded_values = df_copy[col + '_encoded'].unique()
    print(f"Original values for column '{col}': {original_values}")
    print(f"Encoded values for column '{col}': {encoded_values}")
    print()

for col in use_columns:
    df[col] = label_encoder.fit_transform(df[col])

使用LabelEncoder來轉換資料，並列出轉換前後的差異，從結過可得知，Churned(已流失) 0、Joined(已加入) 1、Stayed(已留下) 2

# 相關矩陣(correlation matrix)

In [ ]:
corr = df.corr()

# 打印相關矩陣
print(corr)

In [ ]:
# 與客戶狀態相關
correlation_with_target = corr['客戶狀態']

# 提取相關性數值
positive_correlation_columns = correlation_with_target[correlation_with_target > 0].index.tolist()
positive_correlation_values = correlation_with_target[correlation_with_target > 0].tolist()

# 將相關的列名和相關性數值轉換為 Pandas 的 DataFrame
positive_correlation_info = pd.DataFrame(list(zip(positive_correlation_columns, positive_correlation_values)), columns=['特徵', '相關性'])
print("與客戶狀態呈正相關的特徵及其相關性:")
print(positive_correlation_info)

In [ ]:
# 與客戶狀態相關性數值
correlation_with_target = corr['客戶狀態']

# 提取相關性前15高的特徵及其相關性數值
k = 15
positive_correlation_info = correlation_with_target[correlation_with_target > 0].nlargest(k)

# 將相關的列名和相關性數值轉換為 Pandas 的 DataFrame
positive_correlation_info = positive_correlation_info.reset_index().rename(columns={'index': '特徵', '客戶狀態': '相關性'})

# 列出與客戶狀態呈正相關的前15個特徵及其相關性
print("與客戶狀態呈正相關的前15個特徵及其相關性:")
print(positive_correlation_info)

# 熱力圖(Heatmap)

熱力圖是一種以顏色編碼的方式，用來視覺化數據矩陣中各元素之間相關性的圖表，熱力圖將數據矩陣轉換為直觀的視覺形式，有助於快速理解數據的整體結構還可以通過熱力圖來識別具有高度相關性的特徵，從而進行特徵選擇。

In [ ]:
# 提取相關性前15高的特徵
top_features = positive_correlation_info['特徵']

# 創建相關性矩陣
correlation_matrix = df[top_features].corr()

# 繪製熱力圖
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation Heatmap of Top 15 Features with Highest Positive Correlation to Target (Customer Status)')
plt.xlabel('Features')
plt.ylabel('Features')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Features Selection & Labels Selection

In [ ]:
x = df[['合約類型' ,'加入期間 (月)' ,'推薦次數' ,'總收入' ,'額外長途費用' ,'總費用']]
y = df['客戶狀態']

# Split data

將資料分為80%訓練集和20%測試集

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

# Creating Model

創建了一個決策樹分類模型

In [ ]:
# Creating a DecisionTree Classifier
model = DecisionTreeClassifier(random_state=42)

# GridSearchCV

進行網格搜索來找到最佳的參數組合

In [ ]:
# definitions the range of hyperparameters 定義超參數範圍
parameters_grid = {
    'criterion' :['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Creating a GridSearchCV
grid = GridSearchCV(model, parameters_grid, scoring='accuracy', cv=5)
grid.fit(x_train, y_train)

In [ ]:
# Get best parameters 取得最佳超參數
best_params = grid.best_params_
print("Best Parameters:", best_params)

# Get best model
best_model = grid.best_estimator_
print("Best Model:", best_model)

# Cross-validated score of the best_estimator
print('='*70)
print('Best Training Accuracy Score:', grid.best_score_)

# Predicting
y_pred = best_model.predict(x_test)

# Evaluate

In [ ]:
# Accuracy
print('Accuracy Score : ' ,accuracy_score(y_test ,y_pred))

# Confusion matrix(混淆矩陣)
print('Confusion Matrix : \n' ,confusion_matrix(y_test ,y_pred))

# Classification report(分類報告)
print('Classification Report : \n' ,classification_report(y_test ,y_pred))
# Churned(已流失) 0
# Joined(已加入) 1
# Stayed(已留下) 2

模型在預測「已留下」的類別上表現良好，但在預測「已流失」和「已加入」的類別上的表現相對較差。這可能是由於類別不平衡或特徵選擇的影響。

# Association rule

In [ ]:
df_association = pd.read_csv('/content/drive/MyDrive/大三/上學期/大數據決策/期末報告/customer_data_handled.csv')

In [ ]:
use_columns = ['合約類型', '加入期間 (月)', '推薦次數', '總收入', '額外長途費用', '總費用', '客戶狀態']
df_association = df_association[use_columns]

df_association

In [ ]:
df_association = pd.get_dummies(df_association, columns=['合約類型', '加入期間 (月)', '推薦次數', '總收入', '額外長途費用', '總費用', '客戶狀態'])

df_association

In [ ]:
# 使用Apriori算法找出頻繁項集
frequent_itemsets = apriori(df_association, min_support=0.1, use_colnames=True)

# 生成關聯規則
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

print("頻繁項集：")
print(frequent_itemsets)
print(rules)
rules.to_csv('FR-3_rules.csv', index=False, encoding='utf_8_sig')

第一條規則：如果客戶狀態為「已流失」且推薦次數為0，那麼他們更可能選擇「合約類型」為「Month-to-Month」。這條規則的信賴度為0.92，意味著在這兩個條件同時滿足的情況下，有92%的機會客戶會選擇「Month-to-Month」合約類型，往後可積極推廣長年合約，以增加留存率。

第二條規則：如果合約類型為「Two Year」，那麼客戶更可能是「已留下」的客戶。這條規則的信賴度為0.96，表示當合約類型為「Two Year」時，有96%的機會客戶是「已留下」的，使用長年合約的客戶已留下的機率較高，可將其視為忠誠客戶，並理解客戶的行為模式，並制定相應的業務策略。

# XGboost

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/大三/上學期/大數據決策/期末報告/customer_data_handled.csv')
df.drop(['客戶編號' ,'城市' ,'郵遞區號' ,'緯度' ,'經度'], axis=1, inplace=True)

use_columns = ['性別' ,'婚姻' ,'電話服務' ,'多線路服務' ,'網路服務' ,'線上安全服務' ,'線上備份服務' ,'設備保護計劃' ,'技術支援計劃' ,'電視節目' ,'電影節目' ,'音樂節目' ,'無限資料下載' ,'無紙化計費' ,'優惠方式' ,'網路連線類型','合約類型' ,'支付帳單方式' ,'客戶狀態','客戶流失類別' ,'客戶離開原因']

In [ ]:
# 標籤編碼(LabelEncoder)
label_encoder = LabelEncoder()
labels = ['性別' ,'婚姻' ,'電話服務' ,'多線路服務' ,'網路服務' ,'線上安全服務' ,'線上備份服務' ,'設備保護計劃' ,'技術支援計劃' ,'電視節目' ,'電影節目' ,'音樂節目' ,'無限資料下載' ,'無紙化計費' ,'優惠方式' ,'網路連線類型','合約類型' ,'支付帳單方式' ,'客戶狀態','客戶流失類別' ,'客戶離開原因']
for label in labels:
    label_encoder = LabelEncoder()
    label_encoder.fit(df[label])
    df[label] = label_encoder.transform(df[label])

x = df[['合約類型' ,'加入期間 (月)' ,'推薦次數' ,'總收入' ,'額外長途費用' ,'總費用']]
y = df['客戶狀態']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# Modeling
xbg_model = XGBClassifier(objective='binary:logistic', random_state=42)
xbg_model.fit(x_train, y_train)
y_pred = xbg_model.predict(x_test)
y_train_pred = xbg_model.predict(x_train)

# Accuracy Score
print('Train Accuracy Score : ' ,accuracy_score(y_train ,y_train_pred))
print('Test Accuracy Score : ' ,accuracy_score(y_test ,y_pred))

# Confusion matrix(混淆矩陣)
print('Confusion Matrix : \n' ,confusion_matrix(y_test ,y_pred))

# Classification report(分類報告)
print('Classification Report : \n' ,classification_report(y_test ,y_pred))
print("="*50)
# Churned(已流失) 0
# Joined(已加入) 1
# Stayed(已留下) 2

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001]
}
grid_serch = GridSearchCV(xbg_model, param_grid, cv=5, scoring='accuracy')
grid_serch.fit(x_train, y_train)

print('Best Parameters : ' ,grid_serch.best_params_)
print('Best Score : ' ,grid_serch.best_score_)

best_xgb_model = grid_serch.best_estimator_
y_pred = best_xgb_model.predict(x_test)

# Accuracy Score
print('Accuracy Score : ' ,accuracy_score(y_test ,y_pred))

# Confusion matrix(混淆矩陣)
print('Confusion Matrix : \n' ,confusion_matrix(y_test ,y_pred))

# Classification report(分類報告)
print('Classification Report : \n' ,classification_report(y_test ,y_pred))

透過GridSearchCV調整超參數後，預測Churned(已流失) 0 和 Joined(已加入) 1 的precision都有提升不少，整體accuracy也有些微的上升。